In [1]:
import numpy as np
from tensorflow import keras

max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [8]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [9]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [10]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(layers.Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 128)          2560000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 2,659,074
Trainable params: 2,659,074
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train_one_hot, batch_size=32, epochs=2, validation_split=0.2)

Epoch 1/2
625/625 [==============================] - 54s 84ms/step - loss: 0.4228 - accuracy: 0.8027 - val_loss: 0.3197 - val_accuracy: 0.8696
Epoch 2/2
625/625 [==============================] - 52s 83ms/step - loss: 0.2021 - accuracy: 0.9219 - val_loss: 0.3431 - val_accuracy: 0.8764


In [8]:
model.evaluate(x_test, y_test_one_hot)

782/782 [==============================] - 7s 9ms/step - loss: 0.3909 - accuracy: 0.8218


[0.39088520407676697, 0.8217599987983704]

아래와 같이 stacked bidirectional LSTM을 구현할 수 있다. <br>
이때, return_sequences=True로 설정해야 한다. 

In [9]:
model1 = models.Sequential()
model1.add(layers.Embedding(max_features, 128, input_length=maxlen))
model1.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
model1.add(layers.Bidirectional(layers.LSTM(64)))
model1.add(layers.Dense(2, activation = 'softmax'))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 128)          98816     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 2,757,890
Trainable params: 2,757,890
Non-trainable params: 0
_________________________________________________________________


In [10]:
model1.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model1.fit(x_train, y_train_one_hot, batch_size=32, epochs=2, validation_split=0.2)

Epoch 1/2
782/782 [==============================] - 47s 61ms/step - loss: 0.3833 - accuracy: 0.8268 - val_loss: 0.3378 - val_accuracy: 0.8618
Epoch 2/2
782/782 [==============================] - 46s 59ms/step - loss: 0.2087 - accuracy: 0.9218 - val_loss: 0.3562 - val_accuracy: 0.8607


In [11]:
model1.evaluate(x_test, y_test_one_hot)

782/782 [==============================] - 11s 14ms/step - loss: 0.3562 - accuracy: 0.8607


[0.3561634421348572, 0.8607199788093567]